In [3]:
cd /data/wlsgur4011/part_assembly/src/geotransformer/experiments/lomatch

/data/wlsgur4011/part_assembly/src/geotransformer/experiments/lomatch


In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
sys.path.append("../src/geotransformer/experiments/lomatch")
sys.path.append("../")


import jhutil
import yaml
import torch
torch.manual_seed(1234)
from jhutil import show_meshes

In [7]:
from multi_part_assembly.datasets.geometry_data import build_geometry_dataset, build_geometry_dataloader
from multi_part_assembly.datasets.geometry_data import Sample20k

from config import make_cfg
from torch.utils.data import Dataset, DataLoader
import torch
from jhutil import show_meshes, show_point_clouds

In [8]:

from geotransformer.datasets.registration.threedmatch.dataset import ThreeDMatchPairDataset
from geotransformer.utils.data import (
    registration_collate_fn_stack_mode,
    calibrate_neighbors_stack_mode,
    build_dataloader_stack_mode,
)

def train_valid_data_loader(cfg, distributed, part_assembly=True):

    if part_assembly:
        from part_assembly.stage3 import PairBreakingBadDataset

        datafolder = "/data/wlsgur4011/DataCollection/BreakingBad/data_split/"
        artifact_train = f"{datafolder}artifact.train.pth"
        artifact_val = f"{datafolder}artifact.val.pth"
        train_dataset = PairBreakingBadDataset(artifact_train)
        valid_dataset = PairBreakingBadDataset(artifact_val)
    else:
        train_dataset = ThreeDMatchPairDataset(
            cfg.data.dataset_root,
            'train',
            point_limit=cfg.train.point_limit,
            use_augmentation=cfg.train.use_augmentation,
            augmentation_noise=cfg.train.augmentation_noise,
            augmentation_rotation=cfg.train.augmentation_rotation,
        )
        valid_dataset = ThreeDMatchPairDataset(
            cfg.data.dataset_root,
            'val',
            point_limit=cfg.test.point_limit,
            use_augmentation=False,
        )

    neighbor_limits = calibrate_neighbors_stack_mode(
        train_dataset,
        registration_collate_fn_stack_mode,
        cfg.backbone.num_stages,
        cfg.backbone.init_voxel_size,
        cfg.backbone.init_radius,
    )
    train_loader = build_dataloader_stack_mode(
        train_dataset,
        registration_collate_fn_stack_mode,
        cfg.backbone.num_stages,
        cfg.backbone.init_voxel_size,
        cfg.backbone.init_radius,
        neighbor_limits,
        batch_size=cfg.train.batch_size,
        num_workers=cfg.train.num_workers,
        shuffle=True,
        distributed=distributed,
    )

    valid_loader = build_dataloader_stack_mode(
        valid_dataset,
        registration_collate_fn_stack_mode,
        cfg.backbone.num_stages,
        cfg.backbone.init_voxel_size,
        cfg.backbone.init_radius,
        neighbor_limits,
        batch_size=cfg.test.batch_size,
        num_workers=cfg.test.num_workers,
        shuffle=False,
        distributed=distributed,
    )

    return train_loader, valid_loader, neighbor_limits


In [10]:
cfg = make_cfg()
train_loader, valid_loader, neighbor_limits = train_valid_data_loader(cfg, False, part_assembly=True)

In [11]:

import jhutil; jhutil.jhprint(1111, neighbor_limits)

1111  "array[4] i64 x∈[26, 46] μ=33.500 σ=7.533 [46, 26, 30, 32]"


In [ ]:
import torch
from config import make_cfg
from model import create_model
import sys
sys.path.append("/data/wlsgur4011/part_assembly/src/geotransformer/experiments/lomatch/")
from dataset import train_valid_data_loader


In [ ]:
cfg = make_cfg()
model = create_model(cfg)

state_dict = torch.load("/data/wlsgur4011/BreakingBad/GeoTransformer/weights/geotransformer-3dmatch.pth.tar", map_location=torch.device('cpu'))
model_dict = state_dict['model']
model.load_state_dict(model_dict, strict=False)
model = model.cuda()